In [1]:
# Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Additional Imports
# os - for saving and loading files
# json - to work with json files
# math - to round up results
# time - to add a short pause to not overwhelm the server
import os, json, math, time

# to make yelpapi calls
from yelpapi import YelpAPI

# progress bar from tqdm_notebook
from tqdm.notebook import tqdm_notebook

In [2]:
!pip install yelpapi
!pip install tqdm

In [3]:
# Load API Credentials
with open('C:/Users/david/OneDrive/Documents/May Files/.secret/yelp.json','r') as f:
    login = json.load(f)
login.keys()

dict_keys(['clientid', 'apikey'])

In [4]:
# Instantiate YelpAPI Variable
yelp = YelpAPI(login['apikey'], timeout_s = 5.0)

**Search Terms and File Paths**

In [5]:
# set our API call parameters and filename before the first call
location = 'Tampa, FL 33611'
term = 'vegan'

In [6]:
location.split(',')[0]

'Tampa'

In [7]:
## Specify folder for saving data
FOLDER = 'Data/'

os.makedirs(FOLDER, exist_ok = True)

# Specifying JSON_FILE filename (can include a folder)
JSON_FILE = FOLDER+f"{location.split(',')[0]}-{term}.json"

In [8]:
JSON_FILE

'Data/Tampa-vegan.json'

**Check if Json File exists and Create it if it doesn't**

In [11]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)
## If it does not exist: 
if file_exists == False:    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    
    ## If JSON folder name is not empty:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder, exist_ok = True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f"[i] {JSON_FILE} not found. Saving empty list to file.")
    
    
    ## save the first page of results
    with open(JSON_FILE, 'w') as f:
          json.dump([], f)
        
## If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/Tampa-vegan.json already exists.


In [10]:
os.path.isfile(JSON_FILE)

True

**Initial API Call for first page of Data**

We will use this first result to check:
- How many total results there are?
- Where is the actual data we want to save?
- How many results do we get at a time?

In [12]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp.search_query(term = term, location = location)

In [13]:
type(results)

dict

In [14]:
len(results)

3

In [15]:
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [16]:
results['total']

399

In [17]:
results['region']

{'center': {'longitude': -82.51110076904297, 'latitude': 27.88671929092004}}

In [18]:
results['businesses']

[{'id': 'vyiyaBj88nDjhOb7gR4uDg',
  'alias': 'florish-tampa-bay-2',
  'name': 'Florish',
  'image_url': 'https://s3-media2.fl.yelpcdn.com/bphoto/dTnLaBSbyxPKsFC1s8w47Q/o.jpg',
  'is_closed': False,
  'url': 'https://www.yelp.com/biz/florish-tampa-bay-2?adjust_creative=4nUC1rKElB_Gcs0KiPek7A&utm_campaign=yelp_api_v3&utm_medium=api_v3_business_search&utm_source=4nUC1rKElB_Gcs0KiPek7A',
  'review_count': 80,
  'categories': [{'alias': 'pizza', 'title': 'Pizza'},
   {'alias': 'vegan', 'title': 'Vegan'},
   {'alias': 'vegetarian', 'title': 'Vegetarian'}],
  'rating': 5.0,
  'coordinates': {'latitude': 27.924385, 'longitude': -82.5063638},
  'transactions': ['pickup', 'delivery'],
  'price': '$$',
  'location': {'address1': '2307 S Dale Mabry Hwy',
   'address2': '',
   'address3': '',
   'city': 'Tampa Bay',
   'zip_code': '33629',
   'country': 'US',
   'state': 'FL',
   'display_address': ['2307 S Dale Mabry Hwy', 'Tampa Bay, FL 33629']},
  'phone': '+18132519009',
  'display_phone': '(81

In [19]:
## How many results total?
pd.DataFrame(results['businesses'])

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,vyiyaBj88nDjhOb7gR4uDg,florish-tampa-bay-2,Florish,https://s3-media2.fl.yelpcdn.com/bphoto/dTnLaB...,False,https://www.yelp.com/biz/florish-tampa-bay-2?a...,80,"[{'alias': 'pizza', 'title': 'Pizza'}, {'alias...",5.0,"{'latitude': 27.924385, 'longitude': -82.5063638}","[pickup, delivery]",$$,"{'address1': '2307 S Dale Mabry Hwy', 'address...",+18132519009,(813) 251-9009,4214.018309
1,Cch7lpybfh4V0_1j8Nr3yQ,wevegan-cafe-tampa,Wevegan Cafe,https://s3-media4.fl.yelpcdn.com/bphoto/GGUVcS...,False,https://www.yelp.com/biz/wevegan-cafe-tampa?ad...,36,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {...",4.5,"{'latitude': 27.901782, 'longitude': -82.5183151}",[delivery],NaN,"{'address1': '4326 S Manhattan Ave', 'address2...",,,1807.170948
2,OVMQ5w9Qw96OfZ0e5nZtVA,pure-kitchen-organic-vegan-tampa-2,Pure Kitchen Organic Vegan,https://s3-media4.fl.yelpcdn.com/bphoto/VtB5Zk...,False,https://www.yelp.com/biz/pure-kitchen-organic-...,149,"[{'alias': 'vegan', 'title': 'Vegan'}, {'alias...",5.0,"{'latitude': 27.944519047253, 'longitude': -82...",[delivery],$$,"{'address1': '3214 W Kennedy Blvd', 'address2'...",+18134840558,(813) 484-0558,6582.573303
3,CaS4OzIfDfbYyGai0Wn3Zw,intelligent-gourmet-tampa,Intelligent Gourmet,https://s3-media3.fl.yelpcdn.com/bphoto/CDvf5-...,False,https://www.yelp.com/biz/intelligent-gourmet-t...,60,"[{'alias': 'juicebars', 'title': 'Juice Bars &...",4.5,"{'latitude': 27.912538198162352, 'longitude': ...","[pickup, delivery]",$$,"{'address1': '3413 S Manhattan Ave', 'address2...",+18132872253,(813) 287-2253,2954.416122
4,slO-qEE0z8-BLRTvMYW6HA,the-mediterranean-chickpea-tampa,The Mediterranean Chickpea,https://s3-media3.fl.yelpcdn.com/bphoto/0-BqhY...,False,https://www.yelp.com/biz/the-mediterranean-chi...,224,"[{'alias': 'vegan', 'title': 'Vegan'}, {'alias...",5.0,"{'latitude': 27.9169053561367, 'longitude': -8...",[],$$,"{'address1': '3217 S Macdill Ave', 'address2':...",+18135157981,(813) 515-7981,3758.532994
5,gBKe8wH1TnSmBBpLfHRKbw,bull-market-tampa-tampa,Bull Market Tampa,https://s3-media3.fl.yelpcdn.com/bphoto/zh-I3f...,False,https://www.yelp.com/biz/bull-market-tampa-tam...,131,"[{'alias': 'tacos', 'title': 'Tacos'}, {'alias...",4.5,"{'latitude': 27.8942532, 'longitude': -82.50234}",[delivery],$$,"{'address1': '4807 S Himes Ave', 'address2': '...",+18138982049,(813) 898-2049,1201.295639
6,zwBvdDWFQpYJE1UMDTlSyw,farmacy-vegan-kitchen-and-bakery-tampa,Farmacy Vegan Kitchen & Bakery,https://s3-media3.fl.yelpcdn.com/bphoto/mP-rwt...,False,https://www.yelp.com/biz/farmacy-vegan-kitchen...,313,"[{'alias': 'vegan', 'title': 'Vegan'}, {'alias...",4.5,"{'latitude': 27.95071, 'longitude': -82.46006}","[pickup, delivery]",$$,"{'address1': '803 N Tampa St', 'address2': '',...",+17866811644,(786) 681-1644,8705.101009
7,sieV6f1nJtZkhC1zB9G2eg,wagamama-tampa-tampa-2,wagamama - tampa,https://s3-media2.fl.yelpcdn.com/bphoto/1n8wnj...,False,https://www.yelp.com/biz/wagamama-tampa-tampa-...,139,"[{'alias': 'asianfusion', 'title': 'Asian Fusi...",4.0,"{'latitude': 27.94462, 'longitude': -82.45088}","[pickup, delivery]",$$,"{'address1': '1050 Water St', 'address2': None...",+18134299242,(813) 429-9242,8748.955022
8,CWoQ8o-lyQuQOBSkLQk01g,kale-me-crazy-tampa,Kale Me Crazy,https://s3-media2.fl.yelpcdn.com/bphoto/KaraOW...,False,https://www.yelp.com/biz/kale-me-crazy-tampa?a...,29,"[{'alias': 'juicebars', 'title': 'Juice Bars &...",4.5,"{'latitude': 27.95331853130556, 'longitude': -...","[pickup, delivery]",$$,"{'address1': '1110 Gramercy Ln', 'address2': '...",,,7442.331862
9,er4fflaDW2bRRKy_NfSu8w,market-at-edition-tampa-bay-2,Market at EDITION,https://s3-media3.fl.yelpcdn.com/bphoto/uuIhEB...,False,https://www.yelp.com/biz/market-at-edition-tam...,44,"[{'alias': 'coffee', 'title': 'Coffee & Tea'},...",4.0,"{'latitude': 27.94370181487004, 'longitude': -...",[],$$,"{'address1': '500 Channelside

In [20]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

Calculate how many pages of results needed to cover the total_results

In [21]:
(results['total'])/ results_per_page

19.95

In [22]:
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total'])/ results_per_page)
n_pages

20

In [23]:
for i in tqdm_notebook(range(1,n_pages+1)):
    ## The block of code we want to TRY to run
    try:
        
        time.sleep(.2)
        
        ## Read in results in progress file and check the length
        with open(JSON_FILE, 'r') as f:
            previous_results = json.load(f)
        
        ## save number of results to use as offset
        n_results = len(previous_results)
        
        
        ## use n_results as the OFFSET 
        results = yelp.search_query(location = location, term = term,
                                   offset = n_results+1)

        ## append new results and save to file
        previous_results.extend(results['businesses'])
        
        with open(JSON_FILE, 'w') as f:
            json.dump(previous_results, f)

            
    ## What to do if we get an error/exception.
    except Exception as e:
        print(' [!] ERROR', e)

  0%|          | 0/20 [00:00<?, ?it/s]

**Open JSON File with Pandas**

In [24]:
df = pd.read_json(JSON_FILE)
df.head()

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,location,phone,display_phone,distance,price
0,Cch7lpybfh4V0_1j8Nr3yQ,wevegan-cafe-tampa,Wevegan Cafe,https://s3-media4.fl.yelpcdn.com/bphoto/GGUVcS...,False,https://www.yelp.com/biz/wevegan-cafe-tampa?ad...,36,"[{'alias': 'bakeries', 'title': 'Bakeries'}, {...",4.5,"{'latitude': 27.901782, 'longitude': -82.5183151}",[delivery],"{'address1': '4326 S Manhattan Ave', 'address2...",,,1807.170948,NaN
1,OVMQ5w9Qw96OfZ0e5nZtVA,pure-kitchen-organic-vegan-tampa-2,Pure Kitchen Organic Vegan,https://s3-media4.fl.yelpcdn.com/bphoto/VtB5Zk...,False,https://www.yelp.com/biz/pure-kitchen-organic-...,149,"[{'alias': 'vegan', 'title': 'Vegan'}, {'alias...",5.0,"{'latitude': 27.944519047253, 'longitude': -82...",[delivery],"{'address1': '3214 W Kennedy Blvd', 'address2'...",+18134840558,(813) 484-0558,6582.573303,$$
2,CaS4OzIfDfbYyGai0Wn3Zw,intelligent-gourmet-tampa,Intelligent Gourmet,https://s3-media3.fl.yelpcdn.com/bphoto/CDvf5-...,False,https://www.yelp.com/biz/intelligent-gourmet-t...,60,"[{'alias': 'juicebars', 'title': 'Juice Bars &...",4.5,"{'latitude': 27.912538198162352, 'longitude': ...","[delivery, pickup]","{'address1': '3413 S Manhattan Ave', 'address2...",+18132872253,(813) 287-2253,2954.416122,$$
3,slO-qEE0z8-BLRTvMYW6HA,the-mediterranean-chickpea-tampa,The Mediterranean Chickpea,https://s3-media3.fl.yelpcdn.com/bphoto/0-BqhY...,False,https://www.yelp.com/biz/the-mediterranean-chi...,224,"[{'alias': 'vegan', 'title': 'Vegan'}, {'alias...",5.0,"{'latitude': 27.9169053561367, 'longitude': -8...",[],"{'address1': '3217 S Macdill Ave', 'address2':...",+18135157981,(813) 515-7981,3758.532994,$$
4,gBKe8wH1TnSmBBpLfHRKbw,bull-market-tampa-tampa,Bull Market Tampa,https://s3-media3.fl.yelpcdn.com/bphoto/zh-I3f...,False,https://www.yelp.com/biz/bull-market-tampa-tam...,131,"[{'alias': 'tacos', 'title': 'Tacos'}, {'alias...",4.5,"{'latitude': 27.8942532, 'longitude': -82.50234}",[delivery],"{'address1': '4807 S Himes Ave', 'address2': '...",+18138982049,(813) 898-2049,1201.295639,$$


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Data columns (total 16 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   id             398 non-null    object 
 1   alias          398 non-null    object 
 2   name           398 non-null    object 
 3   image_url      398 non-null    object 
 4   is_closed      398 non-null    bool   
 5   url            398 non-null    object 
 6   review_count   398 non-null    int64  
 7   categories     398 non-null    object 
 8   rating         398 non-null    float64
 9   coordinates    398 non-null    object 
 10  transactions   398 non-null    object 
 11  location       398 non-null    object 
 12  phone          398 non-null    object 
 13  display_phone  398 non-null    object 
 14  distance       398 non-null    float64
 15  price          325 non-null    object 
dtypes: bool(1), float64(2), int64(1), object(12)
memory usage: 47.2+ KB


In [26]:
## convert the filename to a .csv.gz
csv_file = JSON_FILE.replace('.json','.csv.gz')
csv_file

'Data/Tampa-vegan.csv.gz'

In [28]:
## Save it as a compressed csv (to save space)
df.to_csv(csv_file, compression = 'gzip', index = False)

**Compare File size with os module's os.path.getsize**

In [29]:
size_json = os.path.getsize(JSON_FILE)
size_csv_gz = os.path.getsize(JSON_FILE.replace('.json','.csv.gz'))

print(f'JSON FILE: {size_json:,} Bytes')
print(f'CSV.GZ FILE: {size_csv_gz:,} Bytes')

print(f'the csv.gz is {size_json/size_csv_gz} times smaller!')

JSON FILE: 388,558 Bytes
CSV.GZ FILE: 56,676 Bytes
the csv.gz is 6.855776695603077 times smaller!
